# llm helpers

> Helper utilities for using LLMs

In [ ]:
# | default_exp llm.helpers

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

from onprem import utils as U
import json
import yaml
from typing import List, Any

In [ ]:
# | export

def _marshal_llm_to_json(output: str) -> str:
    """
    Extract a substring containing valid JSON or array from a string.

    **Args:**
    
        - output: A string that may contain a valid JSON object or array surrounded by extraneous characters or information.

    **Returns:**
        
        - A string containing a valid JSON object or array.
    """
    output = output.strip()

    left_square = output.find("[")
    left_brace = output.find("{")

    if left_square < left_brace and left_square != -1:
        left = left_square
        right = output.rfind("]")
    else:
        left = left_brace
        right = output.rfind("}")

    return output[left : right + 1]


def parse_json_markdown(text: str) -> Any:
    """
    Parse json embedded in markdown into dictionary
    """
    if "```json" in text:
        text = text.split("```json")[1].strip().strip("```").strip()

    json_string = _marshal_llm_to_json(text)

    try:
        json_obj = json.loads(json_string)
    except json.JSONDecodeError as e_json:
        try:
            # NOTE: parsing again with pyyaml
            #       pyyaml is less strict, and allows for trailing commas
            #       right now we rely on this since guidance program generates
            #       trailing commas
            json_obj = yaml.safe_load(json_string)
        except yaml.YAMLError as e_yaml:
            raise OutputParserException(
                f"Got invalid JSON object. Error: {e_json} {e_yaml}. "
                f"Got JSON string: {json_string}"
            )
        except NameError as exc:
            raise ImportError("Please pip install PyYAML.") from exc

    return json_obj

def parse_code_markdown(text: str, only_last: bool) -> List[str]:
    """
    Parsing embedded code out of markdown string
    """
    # Regular expression pattern to match code within triple-backticks
    pattern = r"```(.*?)```"

    # Find all matches of the pattern in the text
    matches = re.findall(pattern, text, re.DOTALL)

    # Return the last matched group if requested
    code = matches[-1] if matches and only_last else matches

    # If empty we optimistically assume the output is the code
    if not code:
        # we want to handle cases where the code may start or end with triple
        # backticks
        # we also want to handle cases where the code is surrounded by regular
        # quotes
        # we can't just remove all backticks due to JS template strings

        candidate = text.strip()

        if candidate.startswith('"') and candidate.endswith('"'):
            candidate = candidate[1:-1]

        if candidate.startswith("'") and candidate.endswith("'"):
            candidate = candidate[1:-1]

        if candidate.startswith("`") and candidate.endswith("`"):
            candidate = candidate[1:-1]

        # For triple backticks we split the handling of the start and end
        # partly because there can be cases where only one and not the other
        # is present, and partly because we don't need to be so worried
        # about it being a string in a programming language
        if candidate.startswith("```"):
            candidate = re.sub(r"^```[a-zA-Z]*", "", candidate)

        if candidate.endswith("```"):
            candidate = candidate[:-3]
        code = [candidate.strip()]

    return code

In [ ]:
# | export

SUBQUESTION_PROMPT = """\
Given a user question, output a list of relevant sub-questions \
in json markdown that when composed can help answer the full user question.
Only return the JSON response, with no additional text or explanations.

# Example 1
<User Question>
Compare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021

<Output>
```json
{
    "items": [
        {
            "sub_question": "What is the revenue growth of Uber",
        },
        {
            "sub_question": "What is the EBITDA of Uber",
        },
        {
            "sub_question": "What is the revenue growth of Lyft",
        },
        {
            "sub_question": "What is the EBITDA of Lyft",
        }
    ]
}
```

# Example 2
<User Question>
{query_str}

<Output>
"""

def decompose_question(question:str, llm, parse=True):
    """
    Decompose a question into subquestions
    """
    prompt = U.SafeFormatter({'query_str': question}).format(SUBQUESTION_PROMPT)
    json_string = llm.prompt(prompt)
    json_dict = parse_json_markdown(json_string)
    subquestions = [d['sub_question'] for d in json_dict['items']]
    return subquestions
    

In [ ]:
# | export

FOLLOWUP_PROMPT = """\
Given a question, answer "yes" only if the question is complex and follow-up questions are needed or "no" if not.
Always respond with "no" for short questions that are less than 8 words.
Answer only with either "yes" or "no" with no additional text or explanations.

# Example 1
<User Question>
Compare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021

<Output>
yes

# Example 2
<User Question>
How is the Coast Guard using artificial intelligence?

<Output>
No

# Example 3
<User Question
What is AutoGluon?

<Output>
No

# Example 4
<User Question>
{query_str}

<Output>
"""

def needs_followup(question:str, llm, parse=True):
    """
    Decide if follow-up questions are needed
    """
    prompt = U.SafeFormatter({'query_str': question}).format(FOLLOWUP_PROMPT)
    output = llm.prompt(prompt)
    return "yes" in output.lower()

## Examples

In [ ]:
# | notest
from onprem import LLM

In [ ]:
# |  notest

llm = LLM(default_model='llama', n_gpu_layers=-1, verbose=False, mute_stream=True)

llama_new_context_with_model: n_ctx_per_seq (3904) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


### Deciding on Follow-Up Questions

In [ ]:
# |  notest

needs_followup('What is ktrain?', llm=llm)

'no'

In [ ]:
# |  notest

needs_followup('What is the capital of France?', llm=llm)

'no'

In [ ]:
# |  notest

needs_followup("How was Paul Grahams life different before, during, and after YC?", llm)

'yes'

In [ ]:
# |  notest

needs_followup("Compare and contrast the customer segments and geographies of Lyft and Uber that grew the fastest.", llm)

'yes'

In [ ]:
# |  notest

needs_followup("Compare and contrast Uber and Lyft.", llm)

'yes'

### Generating Follow-Up Questions

In [ ]:
# | notest

question = "Compare and contrast the customer segments and geographies of Lyft and Uber that grew the fastest."
subquestions = decompose_question(question, llm=llm, parse=False)
print()
print(subquestions)


['What are the customer segments of Lyft that grew the fastest', 'What are the geographies where Lyft grew the fastest', 'What are the customer segments of Uber that grew the fastest', 'What are the geographies where Uber grew the fastest']


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()